### 简单结构建模

In [2]:
from pymatgen.core.structure import Structure


struct = Structure.from_prototype("bcc", ["Nb"], a=3.32)
# struct = Structure.from_prototype(prototype='CsCl', species=['Cs', 'Cl'], a=4.123)
# struct = Structure.from_prototype(prototype='perovskite', species=['Ca', 'Ti', 'O'], a=3.9)
# struct = Structure.from_prototype(prototype='diamond', species=['Si'], a=5.43)
# struct = Structure.from_prototype(prototype='rocksalt', species=['Na', 'Cl'], a=5.64)
struct

Structure Summary
Lattice
    abc : 3.32 3.32 3.32
 angles : 90.0 90.0 90.0
 volume : 36.594367999999996
      A : 3.32 0.0 0.0
      B : 0.0 3.32 0.0
      C : 0.0 0.0 3.32
    pbc : True True True
PeriodicSite: Nb (0.0, 0.0, 0.0) [0.0, 0.0, 0.0]
PeriodicSite: Nb (1.66, 1.66, 1.66) [0.5, 0.5, 0.5]

In [27]:
# 保存为 VASP 格式文件，默认为 分数坐标
struct.to("Nb.vasp", fmt="poscar")

'Nb2\n1.0\n   3.3199999999999998    0.0000000000000000    0.0000000000000000\n   0.0000000000000000    3.3199999999999998    0.0000000000000000\n   0.0000000000000000    0.0000000000000000    3.3199999999999998\nNb\n2\ndirect\n   0.0000000000000000    0.0000000000000000    0.0000000000000000 Nb\n   0.5000000000000000    0.5000000000000000    0.5000000000000000 Nb\n'

In [28]:
from pymatgen.io.vasp.inputs import Poscar


poscar = Poscar(structure=struct)
# Cartesian 坐标
poscar.write_file("Nb.vasp", direct=False)

In [3]:
# 原胞
struct_primitive = struct.get_primitive_structure()
struct_primitive

Structure Summary
Lattice
    abc : 2.8752043405643364 2.8752043405643364 2.8752043405643364
 angles : 109.47122063449069 109.47122063449069 109.47122063449069
 volume : 18.297183999999998
      A : 1.66 1.66 1.66
      B : 1.66 -1.66 -1.66
      C : -1.66 1.66 -1.66
    pbc : True True True
PeriodicSite: Nb (0.0, 0.0, 0.0) [0.0, 0.0, 0.0]

In [4]:
struct.get_space_group_info()

('Im-3m', 229)

### 晶体结构对称性分析

In [5]:
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer


sga = SpacegroupAnalyzer(struct)

In [9]:
sga.get_symmetry_dataset().keys()

dict_keys(['number', 'hall_number', 'international', 'hall', 'choice', 'transformation_matrix', 'origin_shift', 'rotations', 'translations', 'wyckoffs', 'site_symmetry_symbols', 'crystallographic_orbits', 'equivalent_atoms', 'primitive_lattice', 'mapping_to_primitive', 'std_lattice', 'std_types', 'std_positions', 'std_rotation_matrix', 'std_mapping_to_primitive', 'pointgroup'])

In [7]:
sga.get_symmetry_dataset()  # dict，包含对称性相关的所有信息

{'number': 229,
 'hall_number': 529,
 'international': 'Im-3m',
 'hall': '-I 4 2 3',
 'choice': '',
 'transformation_matrix': array([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]]),
 'origin_shift': array([-2.77555756e-17, -2.77555756e-17, -2.77555756e-17]),
 'rotations': array([[[ 1,  0,  0],
         [ 0,  1,  0],
         [ 0,  0,  1]],
 
        [[-1,  0,  0],
         [ 0, -1,  0],
         [ 0,  0, -1]],
 
        [[ 0, -1,  0],
         [ 1,  0,  0],
         [ 0,  0,  1]],
 
        [[ 0,  1,  0],
         [-1,  0,  0],
         [ 0,  0, -1]],
 
        [[-1,  0,  0],
         [ 0, -1,  0],
         [ 0,  0,  1]],
 
        [[ 1,  0,  0],
         [ 0,  1,  0],
         [ 0,  0, -1]],
 
        [[ 0,  1,  0],
         [-1,  0,  0],
         [ 0,  0,  1]],
 
        [[ 0, -1,  0],
         [ 1,  0,  0],
         [ 0,  0, -1]],
 
        [[ 1,  0,  0],
         [ 0, -1,  0],
         [ 0,  0, -1]],
 
        [[-1,  0,  0],
         [ 0,  1,  0],
         [ 0,  0,  1]],

In [10]:
# wyckoffs letter
sga.get_symmetry_dataset()["wyckoffs"]

['a', 'a']

In [11]:
# 等同原子位点
sga.get_symmetry_dataset()["equivalent_atoms"]

array([0, 0], dtype=int32)

In [8]:
sga.get_crystal_system()

# sga.get_space_group_symbol()
# sga.get_space_group_number()
# sga.get_lattice_type()
# sga.get_space_group_operations()
# sga.get_point_group_symbol()
# sga.get_symmetry_operations()

'cubic'

In [12]:
sga.get_symmetrized_structure()  # 含空间群信息和 Wyckoff site 信息

SymmetrizedStructure
Full Formula (Nb2)
Reduced Formula: Nb
Spacegroup: Im-3m (229)
abc   :   3.320000   3.320000   3.320000
angles:  90.000000  90.000000  90.000000
Sites (2)
  #  SP      a    b    c  Wyckoff
---  ----  ---  ---  ---  ---------
  0  Nb      0    0    0  2a

In [13]:
sga.get_refined_structure()  # 精细结构 通过 spglib.refine_cell() 实现
# refined structure 是指通过对原始结构进行对称性分析和空间群分析后，
# 得到的最小对称性单元的结构。
# 这个过程会将原始结构中的原子重新排列，以便于更好地描述材料的对称性和晶体学性质

Structure Summary
Lattice
    abc : 3.3200000000000003 3.3200000000000003 3.3200000000000003
 angles : 90.0 90.0 90.0
 volume : 36.59436800000001
      A : 3.3200000000000003 0.0 0.0
      B : 0.0 3.3200000000000003 0.0
      C : 0.0 0.0 3.3200000000000003
    pbc : True True True
PeriodicSite: Nb (0.0, 0.0, 0.0) [0.0, 0.0, 0.0]
PeriodicSite: Nb (1.66, 1.66, 1.66) [0.5, 0.5, 0.5]

In [14]:
sga.find_primitive()  # 通过 spglib.find_primitive() 实现

Structure Summary
Lattice
    abc : 2.8752043405643364 2.8752043405643364 2.8752043405643364
 angles : 109.47122063449069 109.47122063449069 109.47122063449069
 volume : 18.297184000000005
      A : 1.6600000000000001 1.6600000000000001 1.6600000000000001
      B : 1.6600000000000001 -1.6600000000000001 -1.6600000000000001
      C : -1.6600000000000001 1.6600000000000001 -1.6600000000000001
    pbc : True True True
PeriodicSite: Nb (0.0, 0.0, 0.0) [0.0, 0.0, 0.0]

In [15]:
sga.get_primitive_standard_structure()

Structure Summary
Lattice
    abc : 2.8752043405643364 2.8752043405643364 2.8752043405643364
 angles : 109.47122063449069 109.47122063449069 109.47122063449069
 volume : 18.297184000000005
      A : -1.6600000000000001 1.6600000000000001 1.6600000000000001
      B : 1.6600000000000001 -1.6600000000000001 1.6600000000000001
      C : 1.6600000000000001 1.6600000000000001 -1.66
    pbc : True True True
PeriodicSite: Nb (0.0, 0.0, 0.0) [0.0, 0.0, 0.0]

In [16]:
sga.get_conventional_standard_structure()

Structure Summary
Lattice
    abc : 3.3200000000000003 3.3200000000000003 3.3200000000000003
 angles : 90.0 90.0 90.0
 volume : 36.59436800000001
      A : 3.3200000000000003 0.0 2.0329136865846065e-16
      B : -2.0329136865846065e-16 3.3200000000000003 2.0329136865846065e-16
      C : 0.0 0.0 3.3200000000000003
    pbc : True True True
PeriodicSite: Nb (0.0, 0.0, 0.0) [0.0, 0.0, 0.0]
PeriodicSite: Nb (1.66, 1.66, 1.66) [0.5, 0.5, 0.5]

In [17]:
sga.get_conventional_to_primitive_transformation_matrix()

array([[-0.5,  0.5,  0.5],
       [ 0.5, -0.5,  0.5],
       [ 0.5,  0.5, -0.5]])

In [19]:
# sga.get_ir_reciprocal_mesh()  # 含义待理解
sga.get_ir_reciprocal_mesh(mesh=(2, 2, 2))

[(array([0., 0., 0.]), 1),
 (array([0.5, 0. , 0. ]), 3),
 (array([0.5, 0.5, 0. ]), 3),
 (array([0.5, 0.5, 0.5]), 1)]

### 从已有构型文件读取结构

In [29]:
from pymatgen.core.structure import Structure

struct = Structure.from_file("Nb.vasp")
struct

Structure Summary
Lattice
    abc : 3.32 3.32 3.32
 angles : 90.0 90.0 90.0
 volume : 36.594367999999996
      A : 3.32 0.0 0.0
      B : 0.0 3.32 0.0
      C : 0.0 0.0 3.32
    pbc : True True True
PeriodicSite: Nb (0.0, 0.0, 0.0) [0.0, 0.0, 0.0]
PeriodicSite: Nb (1.66, 1.66, 1.66) [0.5, 0.5, 0.5]